# Data cleaning and feature engineering

This notebook guides through the process of cleaning the data and extracting meaningful informations as well as feature engineering. 

In [175]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

%matplotlib inline

data_path = "data/sothebys_scraped.csv"
export_path = "data/sothebys_clean.csv"

Let's read in the data and take look at our dataset.

In [176]:
auctions = pd.read_csv(data_path)

In [177]:
auctions.shape

(34514, 6)

In [178]:
auctions[:10]

,car_info,price,additional_info,auction_type,auction_location,lot
0,2017 Jeep Wrangler Custom,"Sold For $57,120",NaN,RM | ONLINE ONLY,ONLINE ONLY: DRIVE INTO THE HOLIDAYS 2019,Lot 107
1,1966 Austin-Healey 3000 Mk III BJ8,"Sold For $58,240",NaN,RM | ONLINE ONLY,ONLINE ONLY: DRIVE INTO THE HOLIDAYS 2019,Lot 106
2,1989 Ferrari Testarossa,Sold After Auction,NaN,RM | ONLINE ONLY,ONLINE ONLY: DRIVE INTO THE HOLIDAYS 2019,Lot 105
3,2018 Audi SQ5,"Sold For $42,560",NaN,RM | ONLINE ONLY,ONLINE ONLY: DRIVE INTO THE HOLIDAYS 2019,Lot 104
4,1960 Austin-Healey 3000 Mk I BN7,"Sold For $40,320",NaN,RM | ONLINE ONLY,ONLINE ONLY: DRIVE INTO THE HOLIDAYS 2019,Lot 103
5,2006 Ford GT,Sold After Auction,NaN,RM | ONLINE ONLY,ONLINE ONLY: DRIVE INTO THE HOLIDAYS 2019,Lot 102
6,1967 Austin Mini Moke,"Sold For $50,400",NaN,RM | ONLINE ONLY,ONLINE ONLY: DRIVE INTO THE HOLIDAYS 2019,Lot 101
7,2009 Mercedes-Benz SL 65 AMG Black Series,"Sold For $161,000",NaN,RM | SOTHEBY'S,ABU DHABI 2019,Lot 103
8,2011 Porsche 911 Speedster,"$300,000 - $350,000",NaN,RM | SOTHEBY'S,ABU DHABI 2019,Lot 104
9,1973 Ferrari 365 GTB/4 Daytona Berlinetta by S...,"Sold For $484,375",NaN,RM | SOTHEBY'S,ABU DHABI 2019,Lot 105


## Initial data cleaning

Our first step in the data cleaning process will be making all the records uniform; we'll make them uppercase.

In [179]:
for col in auctions.columns:
    auctions[col] = auctions[col].str.upper()

Let's start with `car_info` column. Since it's one of the most important ones, we'll have to delete those that are null.

In [180]:
auctions["car_info"].value_counts(dropna=False)

1957 FORD THUNDERBIRD                              124
1957 CHEVROLET BEL AIR                             117
1955 FORD THUNDERBIRD                               77
1966 FORD MUSTANG                                   62
1956 FORD THUNDERBIRD                               62
                                                  ... 
1988 CADILLAC SEDAN DEVILLE                          1
1985 BMW M6                                          1
1976 ROLLS-ROYCE SILVER SHADOW FOUR DOOR SEDAN       1
1932 CHREVROLET FIVE-WINDOW SPORT                    1
1925 BENTLEY 3-LITRE TOURER BY GURNEY NUTTING        1
Name: car_info, Length: 21252, dtype: int64

In [181]:
null_infos = auctions[auctions["car_info"].isnull()]
null_infos

,car_info,price,additional_info,auction_type,auction_location,lot
29087,NaN,SOLD FOR $334,NaN,RM | AUCTIONS,SPRING CARLISLE AUCTION 2011,LOT N001
29088,NaN,SOLD FOR $173,NaN,RM | AUCTIONS,SPRING CARLISLE AUCTION 2011,LOT N002
29089,NaN,SOLD FOR $219,NaN,RM | AUCTIONS,SPRING CARLISLE AUCTION 2011,LOT N003
29090,NaN,SOLD FOR $173,NaN,RM | AUCTIONS,SPRING CARLISLE AUCTION 2011,LOT N004
29091,NaN,SOLD FOR $270,NaN,RM | AUCTIONS,SPRING CARLISLE AUCTION 2011,LOT N005
29092,NaN,SOLD FOR $173,NaN,RM | AUCTIONS,SPRING CARLISLE AUCTION 2011,LOT N011
29093,NaN,SOLD FOR $144,NaN,RM | AUCTIONS,SPRING CARLISLE AUCTION 2011,LOT N014
29094,NaN,SOLD FOR $219,NaN,RM | AUCTIONS,SPRING CARLISLE AUCTION 2011,LOT N015
29095,NaN,SOLD FOR $184,NaN,RM | AUCTIONS,SPRING CARLISLE AUCTION 2011,LOT N020
29096,NaN,NaN,NaN,RM | AUCTIONS,SPRING CARLISLE AUCTION 2011,LOT N026


In [182]:
auctions.drop(null_infos.index, axis=0, inplace=True)

Since we can see lots of null values in `additional_info` column, let's check if this column is worth keeping. We'll also check out `auction_type` column.

In [183]:
((auctions["additional_info"].value_counts(dropna=False))/(auctions.shape[0]))*100

NaN                                                        91.487450
FRIDAY                                                      0.655034
FROM THE ESTATE OF MR. JOHN O'QUINN                         0.553591
OFFERED FROM A PRIVATE COLLECTION                           0.402875
THE YOUNGTIMER COLLECTION                                   0.370993
                                                             ...    
FEATURED IN THE ACADEMY AWARD WINNING FILM "SEABISCUIT"     0.002898
RARE JL8 FOUR-WHEEL DISC BRAKES OPTION                      0.002898
FULLY RESTORED, SHOW QUALITY ICONIC “ROUTE 66” CORVETTE     0.002898
JAGUAR                                                      0.002898
"THE MOST FAMOUS CAR IN THE WORLD"                          0.002898
Name: additional_info, Length: 296, dtype: float64

Around 90% of this column's values are null, we'll drop it from our dataset.

In [184]:
auctions["auction_type"].value_counts(dropna=False)

RM | AUCTIONS       26892
RM | SOTHEBY'S       7597
RM | ONLINE ONLY       13
Name: auction_type, dtype: int64

This column can be useful only to specify location as "Online" for online auctions. It should be deleted afterwards.

In [185]:
online_mask = auctions["auction_type"] == "RM | ONLINE ONLY"
auctions["location"]= np.nan
auctions["location"].mask(online_mask, "Online", inplace=True)

In [186]:
auctions.drop(["additional_info", "auction_type"], axis=1, inplace=True)

Let's check out `lot` column.

In [187]:
auctions["lot"].value_counts(dropna=False)

LOT 123     107
LOT 119     106
LOT 121     106
LOT 124     106
LOT 115     106
           ... 
LOT 7179      1
LOT 7188      1
LOT W777      1
LOT W654      1
LOT 7542      1
Name: lot, Length: 4084, dtype: int64

In [188]:
auctions[auctions["lot"].isnull()]

,car_info,price,auction_location,lot,location


Interestingly, there are no null values, we'll leave this column unchanged and deal with its values in the next step. Let's see `auction_location` now.

In [189]:
auctions["auction_location"].value_counts(dropna=False)

AUBURN FALL 2011                                              1147
AUBURN FALL 2013                                              1126
AUBURN FALL 2014                                              1052
AUBURN FALL 2012                                               972
AUCTIONS AMERICA BY RM - AUBURN COLLECTOR CAR AUCTION 2010     908
                                                              ... 
THE TAJ MA GARAJ COLLECTION                                     32
VILLA D'ESTE 2011                                               31
NEW YORK - DRIVEN BY DISRUPTION 2015                            31
ONLINE ONLY: DRIVE INTO THE HOLIDAYS 2019                        7
NaN                                                              6
Name: auction_location, Length: 164, dtype: int64

In [190]:
auctions[auctions["auction_location"].isnull()== True]

,car_info,price,auction_location,lot,location
47,1991 NISSAN FIGARO,"SOLD FOR $51,750",NaN,LOT,Online
1048,2001 LOTUS ESPRIT V8 SE,"SOLD FOR $40,250",NaN,LOT,Online
1233,1968 MEYERS MANX,"SOLD FOR $55,200",NaN,LOT,Online
1234,2006 FORD GT,"SOLD FOR $310,500",NaN,LOT,Online
1235,2005 PORSCHE 911 TURBO S CABRIOLET,"SOLD FOR $66,125",NaN,LOT,Online
1571,1967 AUSTIN-HEALEY 3000 MK III BJ8,"SOLD FOR $92,000",NaN,LOT,Online


All the null values are online auctions; we'll leave them as they are, since original columns will be deleted anyways.

Let's deal with price column now.

In [201]:
auctions["price"].value_counts(dropna=False, normalize=True)*100

NaN                    4.712770
SOLD FOR $33,000       0.779665
SOLD FOR $22,000       0.718799
SOLD FOR $27,500       0.689815
SOLD FOR $38,500       0.649238
                         ...   
€335,000 - €355,000    0.002898
SOLD FOR €352,000      0.002898
SOLD FOR $54,000       0.002898
SOLD FOR £297,500      0.002898
SOLD FOR €44,850       0.002898
Name: price, Length: 4110, dtype: float64

Around 4.7% of all values don't have any price listed. Since it's too many records to just delete them, we need to look at a potential strategy to impute those; we could:
- impute them with mean or median price from that auction
- impute them with mean or median price for that model (with or without consideration for trend for that model)

## Feature engineering

Since breaking down existing columns into a few more detailed ones is something we'll do many times, it might be a good idea to write a simple function that does it for us. Luckily, there is already a good function for splitting the data, so we only need a function that assigns the data based on the split.

In [202]:
def assign_split_data(dataset, split_data, col_list):
    for split in range(split_data.shape[1]):
        dataset.insert(split, col_list[split], split_data[split])

First, let's break down our data into more columns. We'll start with `car_info`.

In [203]:
auctions["car_info"][:5]

0             2017 JEEP WRANGLER CUSTOM 
1    1966 AUSTIN-HEALEY 3000 MK III BJ8 
2               1989 FERRARI TESTAROSSA 
3                         2018 AUDI SQ5 
4      1960 AUSTIN-HEALEY 3000 MK I BN7 
Name: car_info, dtype: object

Seeing the first 5 entires, we can deduct that it's reasonable to split the column into 4 new columns: 
- year the car was made
- manufacturer (the make of the car)
- the model
- model's variant

We could be more specific but that's something we can easily fix later on when we do the initial analysis.

In [204]:
car_cols = ["year", "manufacturer", "model", "variant"]
car_split = auctions["car_info"].str.split(" ", n=3, expand=True)

In [205]:
assign_split_data(auctions, car_split, car_cols)

In [206]:
auctions.head(5)

,year,manufacturer,model,variant,car_info,price,auction_location,lot,location
0,2017,JEEP,WRANGLER,CUSTOM,2017 JEEP WRANGLER CUSTOM,"SOLD FOR $57,120",ONLINE ONLY: DRIVE INTO THE HOLIDAYS 2019,LOT 107,Online
1,1966,AUSTIN-HEALEY,3000,MK III BJ8,1966 AUSTIN-HEALEY 3000 MK III BJ8,"SOLD FOR $58,240",ONLINE ONLY: DRIVE INTO THE HOLIDAYS 2019,LOT 106,Online
2,1989,FERRARI,TESTAROSSA,,1989 FERRARI TESTAROSSA,SOLD AFTER AUCTION,ONLINE ONLY: DRIVE INTO THE HOLIDAYS 2019,LOT 105,Online
3,2018,AUDI,SQ5,,2018 AUDI SQ5,"SOLD FOR $42,560",ONLINE ONLY: DRIVE INTO THE HOLIDAYS 2019,LOT 104,Online
4,1960,AUSTIN-HEALEY,3000,MK I BN7,1960 AUSTIN-HEALEY 3000 MK I BN7,"SOLD FOR $40,320",ONLINE ONLY: DRIVE INTO THE HOLIDAYS 2019,LOT 103,Online


Now, let's extract the price and the currency, which we will make uniform as a next step.

In [210]:
pattern = r'(?<=SOLD FOR)\s*(.*)\s*'

In [213]:
for row in range(len(auctions)):
    try:
        price = re.search(pattern, str(auctions["price"][row]), re.IGNORECASE).group(1)
        auctions["price_val"][row] = price
    except:
        auctions["price_val"][row] = "Not found"

D:\Anaconda3\envs\data-analysis-env\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [216]:
auctions["price_val"].value_counts(dropna=False)

Not found        9125
$33,000           269
$22,000           248
$27,500           238
$38,500           224
                 ... 
$198,800            1
KR.10,192,000       1
$1,600,000          1
$380,000            1
$213,400            1
Name: price_val, Length: 2955, dtype: int64

Let's drop the original columns.

In [66]:
auctions.drop(["car_info", "price", "auction_location"], axis=1, inplace=True)

Last step is to export the data for others to use.

In [ ]:
# auctions.to_csv(export_path, index = None, header=True)

## Conclusions